In [1]:
import pandas as pd
import numpy as np
import time
from catboost import Pool, cv, CatBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

Импорт данных

In [2]:
features_base = pd.read_csv("features_base.csv", index_col=0).values
features_train = pd.read_csv("features_train.csv", index_col=0).values
target_train = pd.read_csv("target_train.csv", index_col=0).values

In [3]:
idx = pd.read_csv("idx.csv", index_col=0).values
vecs = pd.read_csv("vecs.csv", index_col=0).values
base_index = pd.read_csv("base_index.csv", index_col=0,header=None).to_dict()[1]

Делаем массив 5.000.000 на 145

In [4]:
%%time
idx1 = idx.flatten()[0:5000000]
vecs = vecs.flatten()[0:5000000].reshape(5000000,1)
data1 = features_base[idx1]
data2 = []
#for ind in range(features_train.shape[0]):
for ind in range(10000):    
    data2.extend(np.repeat([features_train[ind]],500,axis=0))
data2 = np.array(data2)  
data = np.hstack((data2,data1,vecs))

Wall time: 37.5 s


Подготавливаем label

In [5]:
label = []
for target, el in zip(target_train[0:10000,0].tolist(), idx[0:10000,:].tolist()):
    for r in el:
       label.append(int(target in base_index[r]))

In [6]:
x_train, x_val, y_train, y_val = train_test_split(data, label, test_size=0.2, random_state=0, shuffle=False)

In [7]:
x_train.shape,x_val.shape

((4000000, 145), (1000000, 145))

In [23]:
train_pool = Pool(x_train, y_train)
val_pool = Pool(x_val, y_val)

In [25]:
grid = {
    'iterations':[500],
    'learning_rate': [0.03],
    'depth':[3],
    #'l2_leaf_reg': [2, 3, 4],
    'boosting_type': ['Ordered', 'Plain']
}

In [27]:
grid_search_model = CatBoostClassifier(
    random_seed=42,    
    auto_class_weights='Balanced',
    loss_function="Logloss")
grid_search_result = grid_search_model.grid_search(grid, train_pool, verbose=20)

0:	learn: 0.6662937	test: 0.6663746	best: 0.6663746 (0)	total: 1.3s	remaining: 10m 47s
1:	learn: 0.6422478	test: 0.6422025	best: 0.6422025 (1)	total: 2.95s	remaining: 12m 15s
2:	learn: 0.6232169	test: 0.6230149	best: 0.6230149 (2)	total: 4.16s	remaining: 11m 28s
3:	learn: 0.6052569	test: 0.6049262	best: 0.6049262 (3)	total: 5.66s	remaining: 11m 41s
4:	learn: 0.5841006	test: 0.5837307	best: 0.5837307 (4)	total: 7.09s	remaining: 11m 42s
5:	learn: 0.5691773	test: 0.5688236	best: 0.5688236 (5)	total: 8.39s	remaining: 11m 31s
6:	learn: 0.5497912	test: 0.5494610	best: 0.5494610 (6)	total: 9.71s	remaining: 11m 24s
7:	learn: 0.5369763	test: 0.5365903	best: 0.5365903 (7)	total: 11s	remaining: 11m 15s
8:	learn: 0.5227716	test: 0.5221902	best: 0.5221902 (8)	total: 12.4s	remaining: 11m 17s
9:	learn: 0.5086710	test: 0.5081657	best: 0.5081657 (9)	total: 13.7s	remaining: 11m 12s
10:	learn: 0.4962462	test: 0.4955130	best: 0.4955130 (10)	total: 15.1s	remaining: 11m 12s
11:	learn: 0.4837533	test: 0.4830

clf = CatBoostClassifier(
    iterations=500,
    random_seed=42,
    learning_rate=0.2,
    depth=3,
    loss_function="Logloss",
    #eval_metric='AUC',
    auto_class_weights='Balanced'
)


clf.fit(
    x_train, y_train,      
    verbose=True   
)

In [ ]:
def calc_accuracy(train_pool=train_pool, val_pool=val_pool, **kwargs):
    model = catboost.CatBoostClassifier(**kwargs, random_seed=42)
    model.fit(train_pool, verbose=0, eval_set=val_pool)
    y_pred = model.predict_proba(test_pool)
    return eval_metric(test_pool.get_label(), y_pred[:, 1], 'AUC')

In [95]:
predictions = clf.predict(x_val)

In [89]:
accuracy_score(predictions,y_val)

0.959976

In [21]:
proba = clf.predict_proba(x_val)[:,1]
proba1 = proba.reshape(2000,500)
y_val1 = np.array(y_val).reshape(2000,500)
acc5 = 0
for val,prob in zip(y_val1,proba1):
    acc5 += np.sum(val[np.argsort(prob)[-5:]])
acc5 = acc5/len(y_val1)
acc5

0.7285

In [22]:
proba = clf.predict_proba(x_train)[:,1]
proba1 = proba.reshape(8000,500)
y_val1 = np.array(y_train).reshape(8000,500)
acc5 = 0
for val,prob in zip(y_val1,proba1):
    acc5 += np.sum(val[np.argsort(prob)[-5:]])
acc5 = acc5/len(y_val1)
acc5

0.7675